# Tabular data - 5-way classifier
# Stratified KFold implementation. 
 
 
Code to incorporate Stratified KFold has been added to the copy of the single run script 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from string import ascii_letters
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.datasets import make_classification  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.preprocessing import StandardScaler, PowerTransformer
import tensorflow as tf
from tensorflow.random import set_seed
set_seed(42)
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import PowerTransformer
from scipy import optimize
import os

In [ ]:
img_dpi= 300

In [ ]:

from sinfo import sinfo
sinfo()

# Set paths

## Results paths

In [ ]:
path_res = Path(os.path.abspath("../demo_output/tab_data_5way_skfold/"))

In [ ]:
path_plots = path_res.joinpath("Plots")
print(path_plots)
path_plots.is_dir()

In [ ]:
#Added code to set the path to save the model
path_models = path_res.joinpath("saved_models")
print(path_models)
path_models.is_dir()

In [ ]:
#Added code to create folders if not present
if not path_res.is_dir():
    path_res.mkdir(parents=True, exist_ok=True)

if not path_plots.is_dir():
    path_plots.mkdir(parents=True, exist_ok=True)
    
if not path_models.exists():
    path_models.mkdir(parents=True, exist_ok=True)

## Read data

In [ ]:

data_file = Path(os.path.abspath("../demo_data/Tabular_subset.csv"))

print(data_file)


## Load data


In [ ]:
df1 = pd.read_csv (data_file)

In [ ]:
display(df1.head())
df1.shape


In [ ]:
list(df1.columns)

In [ ]:
df_features = df1.iloc[:, 1:41]

In [ ]:
df_features.head()

In [ ]:
df_features.shape

In [ ]:
corr = df_features.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(35, 25))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, annot = True, fmt='.2f', mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
df1["label"] = df1.label.astype('category')

In [ ]:
df1.insert(0, 'pathway', df1.label.cat.codes)

df1.head()

In [ ]:
all_cols = list(df1)
len(all_cols)

In [ ]:
all_feats = [i for i in all_cols if i.startswith('Live')]
len(all_feats)

In [ ]:
all_cols

In [ ]:
mask = df1[all_cols].isnull().sum(axis=1)

tmp = pd.DataFrame(
#     pd.concat([
    pd.DataFrame(mask[mask>0], columns=["number_of_points"])["number_of_points"].value_counts().reset_index().rename(
        columns={'index':'number_of_features_missing'}
    ).set_index("number_of_points"), 
)

tmp['percentage_feature_missing'] = np.round(tmp['number_of_features_missing']/len(all_cols)*100., decimals=2)
tmp

In [ ]:
mask = df1[all_feats].isnull().sum(axis=1)

to_remove = pd.DataFrame(mask[mask>=20], columns=["number_of_points"]) # these are the five points that need to be dropped.

to_remove

In [ ]:
df1.shape

In [ ]:
df1 = df1.drop(index=to_remove.index)
df1.shape

In [ ]:
plate_cols = ['PlateID']

In [ ]:
label_cols = ['label', 'label_name', 'pathway']

In [ ]:
class SafePowerTransformer(PowerTransformer):
    def _yeo_johnson_optimize(self, x):
        def _neg_log_likelihood(lmbda):
            x_trans = self._yeo_johnson_transform(x, lmbda)
            n_samples = x.shape[0]
            variance = x_trans.var()
            if variance == 0:
                return np.inf  
            loglike = -n_samples / 2 * np.log(variance) 
            loglike += (lmbda - 1) * (np.sign(x) * np.log1p(np.abs(x))).sum()
            return -loglike
        x = x[~np.isnan(x)]
        return optimize.brent(_neg_log_likelihood, brack=(-2, 2))

In [ ]:
df1.label.unique()

In [ ]:
def evaluate_model(data, all_features, n_folds, fname_prefix_save_models):
    #data = df1
    #n_folds = 10
    #all_features= all_feats
    
    X = data
    y = data.label

    scores, histories = list(), list()
    fold_ct = 0
    # prepare cross validation
    skf = StratifiedKFold(n_splits= n_folds, shuffle=True, random_state=1)
    for train_index, test_index in skf.split(X, y):
        fold_ct+=1
        print("Fold", fold_ct)
        df_train, df_test = X.iloc[train_index], X.iloc[test_index]
        df_train, df_val = train_test_split(df_train,   
                                          random_state=42, stratify=df_train.label)

        imp = IterativeImputer(max_iter=10, random_state=0, initial_strategy='median')
        lambdas = {}

        scaled_df_train = []
        for eachplate in df_train.PlateID.unique():
            dd = df_train[df_train.PlateID==eachplate].copy()

            # fit for scaling. Scale per control per plate, then apply the transformation to all datapoints in the plate.
            scaler = SafePowerTransformer()
            scaler.fit(dd[dd.label==4][all_features].values)

            # scale
            dd.loc[:,all_features] = scaler.transform(dd[all_features].values)

            # inpute missing values using iterative inputer
            dd.loc[:,all_features] = imp.fit_transform(dd[all_features].values)

            lambdas[eachplate] = scaler.lambdas_
            scaled_df_train.append(dd)

        scaled_df_train = pd.concat(scaled_df_train)
        scaled_df_train = scaled_df_train.sample(frac = 1, random_state=200) #shuffle - added

        scaled_df_test = []
        for eachplate in df_test.PlateID.unique():
            dd = df_test[df_test.PlateID==eachplate].copy()

            # fit for scaling. Scale per control per plate, then apply the transformation to all datapoints in the plate.
            scaler = SafePowerTransformer()
            scaler.fit(dd[dd.label==4][all_features].values)

            # scale
            dd.loc[:,all_features] = scaler.transform(dd[all_features].values)

            # inpute missing values using iterative inputer
            dd.loc[:,all_features] = imp.fit_transform(dd[all_features].values)

            lambdas[eachplate] = scaler.lambdas_
            scaled_df_test.append(dd)

        scaled_df_test = pd.concat(scaled_df_test)
        scaled_df_test = scaled_df_test.sample(frac = 1, random_state=200) #shuffle - added

        scaled_df_val = []
        for eachplate in df_val.PlateID.unique():
            dd = df_val[df_val.PlateID==eachplate].copy()

            # fit for scaling. Scale per control per plate, then apply the transformation to all datapoints in the plate.
            scaler = SafePowerTransformer()
            scaler.fit(dd[dd.label==4][all_features].values)

            # scale
            dd.loc[:,all_features] = scaler.transform(dd[all_features].values)

            # inpute missing values using iterative inputer
            dd.loc[:,all_features] = imp.fit_transform(dd[all_features].values)

            lambdas[eachplate] = scaler.lambdas_
            scaled_df_val.append(dd)

        scaled_df_val = pd.concat(scaled_df_val)
        scaled_df_val = scaled_df_val.sample(frac = 1, random_state=200) #shuffle - added

        #scaled dds
        scaled_dds_train = []
        for eachp in scaled_df_train.PlateID.unique():
            dd = scaled_df_train[scaled_df_train.PlateID==eachp].sample(n=scaled_df_train.PlateID.value_counts().min())
            scaled_dds_train.append(dd)
        scaled_dds_train = pd.concat(scaled_dds_train).sample(frac=1).reset_index(drop=True)

        scaled_dds_val = []
        for eachp in scaled_df_val.PlateID.unique():
            dd = scaled_df_val[scaled_df_val.PlateID==eachp].sample(n=scaled_df_val.PlateID.value_counts().min())
            scaled_dds_val.append(dd)
        scaled_dds_val = pd.concat(scaled_dds_val).sample(frac=1).reset_index(drop=True)

        scaled_dds_test = []
        for eachp in scaled_df_test.PlateID.unique():
            dd = scaled_df_test[scaled_df_test.PlateID==eachp].sample(n=scaled_df_test.PlateID.value_counts().min())
            scaled_dds_test.append(dd)
        scaled_dds_test = pd.concat(scaled_dds_test).sample(frac=1).reset_index(drop=True)

        # end scaled dds

        x_train = scaled_df_train[all_features]
        x_val = scaled_df_val[all_features]
        x_test = scaled_df_test[all_features]

        y_train = scaled_df_train['label'].values
        y_val = scaled_df_val['label'].values
        y_test = scaled_df_test['label'].values

        scal_y_train_onehot = to_categorical(y_train)
        scal_y_val_onehot = to_categorical(y_val)
        scal_y_test_onehot = to_categorical(y_test)

        scal_model_name = fname_prefix_save_models.parent.joinpath(os.path.basename(fname_prefix_save_models) + f"tab_5way_powertrans_plate-scaling_dense-40_3_fold{fold_ct}")

        scal_checkpoint = ModelCheckpoint(
        str(scal_model_name)+'.hdf5', 
        monitor='val_loss', 
        verbose=0,
        save_best_only=True, 
        mode='auto', 
        save_freq='epoch'
        )

        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            min_delta=0, 
            patience=50, 
            verbose=0, 
            mode='auto',
            baseline=None, 
            restore_best_weights=True,
        )
        dropout_rate = 0.1

        input_t = Input(shape=(x_train.shape[1],))

        # x = Dense(10, activation='relu')(input_t)

        x = Dense(40, activation='relu')(input_t)
        x = Dropout(dropout_rate)(x)
        x = Dense(40, activation='relu')(x)
        x = Dropout(dropout_rate)(x)
        x = Dense(40, activation='relu')(x)
        x = Dropout(dropout_rate)(x)


        output = Dense(scal_y_train_onehot.shape[1], activation='softmax')(x)

        scal_model = Model(inputs=input_t, outputs = output)
        scal_model.compile(
            loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer='adam',
        )

        batch_size = 100
        epochs = 1000
        history = scal_model.fit(
            x_train, 
            scal_y_train_onehot,
            validation_data = (x_val, scal_y_val_onehot),
            batch_size = batch_size,
            epochs = epochs,
            callbacks=[scal_checkpoint, early_stop],
            verbose=0
        )

         # evaluate model- load the saved model
        scal_model_saved = tf.keras.models.load_model(str(scal_model_name)+'.hdf5')
        _, acc = scal_model_saved.evaluate(x_test, scal_y_test_onehot, verbose=0)
        print('> %.3f' % (acc * 100.0))

         # stores scores
        scores.append(acc)
        histories.append(history)
    return scores, histories

In [ ]:
path_models.joinpath("mixed-plates_")

In [ ]:
path_prefix_model = path_models.joinpath("mixed-plates_")

#Evaluate the model - makes and evaluates the model 
scores, histories = evaluate_model(data=df1, all_features=all_feats, n_folds=10,  fname_prefix_save_models= path_prefix_model )

    

In [ ]:
def summarize_diagnostics(histories, title, path_save_plot_prefix, img_dpi=300 ):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,7))
    for i in range(len(histories)):
        ax1.plot(histories[i].history['loss'], color='blue',  label='Training')
        ax1.plot(histories[i].history['val_loss'], color='orange', label='Validation')
    
        ax2.plot(histories[i].history['accuracy'], color='blue', label='Training')
        ax2.plot(histories[i].history['val_accuracy'], color='orange', label='Validation')
    
    ax1.set_title("Loss")
    ax1.set_ylabel("Loss")
    ax1.set_xlabel("Epochs")
    ax1.legend(frameon=False, labels = ["Training", "Validation"])
    
    ax2.set_title("Accuracy")
    ax2.set_ylabel("Accuracy")
    ax2.set_xlabel("Epochs")
    ax2.legend(frameon=False, labels = ["Training", "Validation"])

    fig.suptitle(title);
    fig.savefig(path_save_plot_prefix.parent.joinpath(os.path.basename(path_save_plot_prefix)+"_Model-perform.png"), bbox_inches='tight', dpi = img_dpi)
    
    return 

In [ ]:
#saving the accuracy scores 
open_file = open(path_models.joinpath("mixed-plates_tab_5way_accuracy_scores.pkl"), "wb")
pickle.dump(scores, open_file)
open_file.close()

# Plot model histories and scores

In [ ]:
plot_title = "Tabular data - Predicting disease pathways and controls across plates, Power transform per control within plate"
path_save_plot = path_plots.joinpath(os.path.basename(path_prefix_model))
summarize_diagnostics(histories, title=plot_title, path_save_plot_prefix = path_save_plot)

In [ ]:
def summarize_scores(scores, title ):
    #print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (np.mean(scores)*100, np.std(scores)*100, len(scores)))
    # box and whisker plots of results
    plt.boxplot(scores)
    plt.suptitle(title)
    plt.show()
    return

In [ ]:
plot_title = ("Tabular data - Predicting disease pathways and controls across plates,"
                              "\n Accuracy")
summarize_scores(scores, plot_title )